In [1]:
import pandas as pd
import optuna.integration.lightgbm as lgb
import optuna

In [2]:
de_train = pd.read_parquet(f"./train/de_train.parquet")
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [4]:
from sklearn.preprocessing import OneHotEncoder
from rdkit import Chem
from rdkit.Chem import RDKFingerprint
import numpy as np

In [5]:
cell_types = de_train.iloc[:,0:1]

In [6]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(cell_types)
cell_types_ohe = ohe.transform(cell_types)
cell_types_ohe = cell_types_ohe.tolist()
de_train['cell_type_ohe'] = cell_types_ohe
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_ohe
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"


In [7]:
perts = de_train["SMILES"]
print(len(perts))
perts = perts.dropna()
print(len(perts))

614
614


In [8]:
cs_arr = []
for pert in perts:
  try:
    cs = Chem.CanonSmiles(pert)
    cs_arr.append(cs)
  except:
    print('Invalid SMILES: ', pert)
print(len(cs_arr))

614


In [9]:
mols = [Chem.MolFromSmiles(x) for x in cs_arr]
fps = [np.array(RDKFingerprint(mol)) for mol in mols]
de_train['fingerprint'] = fps
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_ohe,fingerprint
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


In [102]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import sklearn

In [11]:
res_arr = []
for i in range(len(de_train)):
  ct_ohe = np.array(de_train.iloc[i,-2:-1][0])
  fingerprint = np.array(de_train.iloc[i,-1:][0])
  res = np.concatenate((ct_ohe, fingerprint), axis=0)
  res_arr.append(res)

In [12]:
de_train['merged'] = res_arr

In [13]:
b_cells_df = de_train[de_train["cell_type"] == "B cells"]
my_cells_df = de_train[de_train["cell_type"] == "Myeloid cells"]

In [33]:
b_cells_idxes = list(b_cells_df.index)
my_cells_idxes = list(my_cells_df.index)

In [34]:
tmp_x = de_train.iloc[:,-1:]
arr = []
for i in range(len(tmp_x)):
    if i in b_cells_idxes or i in my_cells_idxes:
        continue
    arr.append(tmp_x.iloc[i,0])
X = np.array(arr)
print(X.shape)

(580, 2054)


In [133]:
tmp_y = de_train.iloc[:,5:-3]
#tmp_y = de_train.iloc[:,5:105]
arr = []
for i in range(len(tmp_y)):
    if i in b_cells_idxes or i in my_cells_idxes:
        continue
    arr.append(tmp_y.iloc[i].values)
Y = np.array(arr)
print(Y.shape)

(580, 18211)


Repeat the same w/ B Cells and Myeloid Cells.

In [135]:
tmp_x = de_train.iloc[:,-1:]
arr = []
for i in range(len(tmp_x)):
    if i in b_cells_idxes:
        arr.append(tmp_x.iloc[i,0])
X_b = np.array(arr)
print(X_b.shape)

#tmp_y = de_train.iloc[:,5:105]
tmp_y = de_train.iloc[:,5:-3]
arr = []
for i in range(len(tmp_y)):
    if i in b_cells_idxes:
        arr.append(tmp_y.iloc[i].values)
Y_b = np.array(arr)
print(Y_b.shape)

(17, 2054)
(17, 18211)


In [136]:
tmp_x = de_train.iloc[:,-1:]
arr = []
for i in range(len(tmp_x)):
    if i in my_cells_idxes:
        arr.append(tmp_x.iloc[i,0])
X_my = np.array(arr)
print(X_my.shape)

#tmp_y = de_train.iloc[:,5:105]
tmp_y = de_train.iloc[:,5:-3]
arr = []
for i in range(len(tmp_y)):
    if i in my_cells_idxes:
        arr.append(tmp_y.iloc[i].values)
Y_my = np.array(arr)
print(Y_my.shape)

(17, 2054)
(17, 18211)


In [137]:
def mrrmse_np(y_pred, y_true):
    return np.sqrt(np.square(y_true - y_pred).mean(axis=1)).mean()

In [122]:
def objective(trial):
    train_x_split, test_x_split, train_y_split, test_y_split = train_test_split(X, Y, test_size=0.2, random_state=42) 
    train_x, val_x, train_y, val_y = train_test_split(train_x_split, train_y_split, test_size=0.2, random_state=42)
    
    train_xb, test_xb, train_yb, test_yb = train_test_split(X_b, Y_b, test_size=0.8, random_state=42)
    train_xb, val_xb, train_yb, val_yb = train_test_split(train_xb, train_yb, test_size=0.2, random_state=42)
    train_xmy, test_xmy, train_ymy, test_ymy = train_test_split(X_my, Y_my, test_size=0.8, random_state=42)
    train_xmy, val_xmy, train_ymy, val_ymy = train_test_split(train_xmy, train_ymy, test_size=0.2, random_state=42)
    
    X_train = np.concatenate((train_xb, train_xmy, train_x), axis=0)
    y_train = np.concatenate((train_yb, train_ymy, train_y), axis=0)
    X_val = np.concatenate((val_xb, val_xmy, val_x), axis=0)
    y_val = np.concatenate((val_yb, val_ymy, val_y), axis=0) 
    X_test = np.concatenate((test_xb, test_xmy, test_x_split), axis=0)
    y_test = np.concatenate((test_yb, test_ymy, test_y_split), axis=0)
    
    # X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, Y, test_size=0.2, random_state=42)
    # X_train, X_val, y_train, y_val = train_test_split(X_train_split, y_train_split, test_size=0.25, random_state=42)
    
    models = []
    
    for i in tqdm(range(Y.shape[1])):
        param = {
        'objective': 'regression',
        'metric': 'l2',
        'verbose': -1,
        'num_threads': 8,
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 15.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 15.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 1, 5),
        'num_iterations': trial.suggest_int('num_iterations', 50, 500),
        'path_smooth': trial.suggest_float('path_smooth', 0, 1),
        'num_boost_round': trial.suggest_int('num_boost_round', 2, 100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 5),
        'extra_trees': True,
        'min_sum_hessian_in_leaf': trial.suggest_int('min_sum_hessian_in_leaf', 2, 5),
        }
        dtrain = lgb.Dataset(X_train, label=y_train[:, i])
        dval = lgb.Dataset(X_val, label=y_val[:, i], reference=dtrain) 
        gbm = lgb.train(param, dtrain, valid_sets=[dval])
        models.append(gbm)
        
    pred_y = np.array([model.predict(X_test) for model in models]).T
    loss = mrrmse_np(pred_y, y_test)
    print(loss)
    return loss

In [17]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [129]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
 
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

  0%|          | 0/100 [00:00<?, ?it/s]/scratch4/mschatz1/hji20/lgbm/lib/python3.8/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/scratch4/mschatz1/hji20/lgbm/lib/python3.8/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
100%|██████████| 100/100 [00:13<00:00,  7.41it/s]


1.6258522526976307


100%|██████████| 100/100 [00:26<00:00,  3.80it/s]


1.593312398512948


100%|██████████| 100/100 [00:20<00:00,  4.86it/s]


...

1.5721833403632228


100%|██████████| 100/100 [00:21<00:00,  4.75it/s]


1.576289216758041


100%|██████████| 100/100 [00:15<00:00,  6.29it/s]

1.5904037557842527
Number of finished trials: 100
Best trial: {'lambda_l1': 14.657361225174188, 'lambda_l2': 13.39489197368847, 'num_leaves': 13, 'feature_fraction': 0.42552360266723904, 'bagging_fraction': 0.9488006401790557, 'learning_rate': 0.1767596522647403, 'bagging_freq': 8, 'min_child_samples': 19, 'early_stopping_rounds': 5, 'num_iterations': 68, 'path_smooth': 0.9684876288861394, 'num_boost_round': 25, 'min_data_in_leaf': 4, 'min_sum_hessian_in_leaf': 4}


In [130]:
print('Best trial:', study.best_trial.params)

Best trial: {'lambda_l1': 14.657361225174188, 'lambda_l2': 13.39489197368847, 'num_leaves': 13, 'feature_fraction': 0.42552360266723904, 'bagging_fraction': 0.9488006401790557, 'learning_rate': 0.1767596522647403, 'bagging_freq': 8, 'min_child_samples': 19, 'early_stopping_rounds': 5, 'num_iterations': 68, 'path_smooth': 0.9684876288861394, 'num_boost_round': 25, 'min_data_in_leaf': 4, 'min_sum_hessian_in_leaf': 4}


In [117]:
train_x_split, test_x_split, train_y_split, test_y_split = train_test_split(X, Y, test_size=0.2, random_state=42) 
train_x, val_x, train_y, val_y = train_test_split(train_x_split, train_y_split, test_size=0.2, random_state=42)
    
train_xb, test_xb, train_yb, test_yb = train_test_split(X_b, Y_b, test_size=0.8, random_state=42)
train_xb, val_xb, train_yb, val_yb = train_test_split(train_xb, train_yb, test_size=0.2, random_state=42)
train_xmy, test_xmy, train_ymy, test_ymy = train_test_split(X_my, Y_my, test_size=0.8, random_state=42)
train_xmy, val_xmy, train_ymy, val_ymy = train_test_split(train_xmy, train_ymy, test_size=0.2, random_state=42)
    
X_train = np.concatenate((train_xb, train_xmy, train_x), axis=0)
y_train = np.concatenate((train_yb, train_ymy, train_y), axis=0)
X_val = np.concatenate((val_xb, val_xmy, val_x), axis=0)
y_val = np.concatenate((val_yb, val_ymy, val_y), axis=0) 
X_test = np.concatenate((test_xb, test_xmy, test_x_split), axis=0)
y_test = np.concatenate((test_yb, test_ymy, test_y_split), axis=0)

Optimized:

In [131]:
models_opt = []
    
for i in tqdm(range(Y.shape[1])):
    param = study.best_trial.params
    param['verbosity'] = -1
    dtrain = lgb.Dataset(X_train, label=y_train[:, i])  # Train a model for each output element
    dval = lgb.Dataset(X_val, label=y_val[:, i], reference=dtrain) 
    gbm = lgb.train(param, dtrain, valid_sets=[dval])
    models_opt.append(gbm)
        
pred_y = np.array([model.predict(X_test) for model in models_opt]).T
loss = mrrmse_np(pred_y, y_test)
loss2 = sklearn.metrics.mean_squared_error(pred_y, y_test)
print(loss)
print(loss2)

100%|██████████| 100/100 [00:19<00:00,  5.12it/s]


1.5655951835918471
7.403258923269011


Unoptimized:

In [132]:
models_def = []
    
for i in tqdm(range(Y.shape[1])):
    param = {
        'objective': 'regression',
        'metric': 'l2',
        'num_leaves': 100,
        'learning_rate': 0.1,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'num_threads': 8
    }
    dtrain = lgb.Dataset(X_train, label=y_train[:, i])  # Train a model for each output element
    dval = lgb.Dataset(X_val, label=y_val[:, i], reference=dtrain) 
    gbm = lgb.train(param, dtrain, valid_sets=[dval])
    models_def.append(gbm)
        
pred_y = np.array([model.predict(X_test) for model in models_def]).T
loss = mrrmse_np(pred_y, y_test)
loss2 = sklearn.metrics.mean_squared_error(pred_y, y_test)
print(loss)
print(loss2)

100%|██████████| 100/100 [00:20<00:00,  4.88it/s]

1.6959456709412073
7.365390995882534


In [31]:
saved_models = []
for i in range(100):
    fn = "trained_models/trained_model_" + str(i) + ".txt"
    model = lgb.Booster(model_file=fn)
    saved_models.append(model)

Saved models from latest Kaggle submission:

In [124]:
pred_y = np.array([model.predict(X_test) for model in saved_models]).T
loss = mrrmse_np(pred_y, y_test)
loss2 = sklearn.metrics.mean_squared_error(pred_y, y_test)
print(loss)
print(loss2)

1.4822118051256534
6.515382869435226


# Prepare Predictions

In [140]:
id_map = pd.read_csv("./train/id_map.csv", sep=",", dtype=str)
id_map.columns
id_map.head()

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867


In [141]:
ohe_col = []
for x in id_map.cell_type:
  if x == "B cells":
    ohe_col.append([1.0, 0.0, 0.0, 0.0, 0.0, 0.0])
  else:
    ohe_col.append([0.0, 1.0, 0.0, 0.0, 0.0, 0.0])
id_map["ohe_col"] = ohe_col
id_map.head()

,id,cell_type,sm_name,ohe_col
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,1,B cells,ABT-199 (GDC-0199),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,2,B cells,ABT737,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,3,B cells,AMD-070 (hydrochloride),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,4,B cells,AT 7867,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [142]:
cp_df = de_train[["sm_name", "SMILES"]].drop_duplicates()
cp_df.head()

,sm_name,SMILES
0,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1
4,Mometasone Furoate,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...
8,Idelalisib,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...
14,Vandetanib,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1
18,Bosutinib,COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC)cc...


In [143]:
cs_arr = []
sm_arr = cp_df["SMILES"]
for sm in sm_arr:
  try:
    cs = Chem.CanonSmiles(sm)
    cs_arr.append(cs)
  except:
    print('Invalid SMILES: ', sm)
print(len(cs_arr))

146


In [145]:
mols = [Chem.MolFromSmiles(x) for x in cs_arr]
fps = [np.array(RDKFingerprint(mol)) for mol in mols]
cp_df['fingerprint'] = fps
cp_df["fingerprint"] = cp_df["fingerprint"].apply(lambda x: np.array(x))
cp_df.head()

,sm_name,SMILES,fingerprint
0,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,"[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,Mometasone Furoate,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,"[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."
8,Idelalisib,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, ..."
14,Vandetanib,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,"[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, ..."
18,Bosutinib,COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC)cc...,"[1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, ..."


In [148]:
id_map_merged = pd.merge(id_map, cp_df, on="sm_name", how='inner')
assert len(id_map) == len(id_map_merged)
id_map_merged.head()

,id,cell_type,sm_name,ohe_col,SMILES,fingerprint
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ..."
1,128,Myeloid cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ..."
2,1,B cells,ABT-199 (GDC-0199),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
3,129,Myeloid cells,ABT-199 (GDC-0199),"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
4,2,B cells,ABT737,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...,"[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."


In [149]:
res_arr = []
for i in range(len(id_map_merged)):
  ct_ohe = np.array(id_map_merged.iloc[i,-3:-2][0])
  fingerprint = np.array(id_map_merged.iloc[i,-1:][0])
  res = np.concatenate((ct_ohe, fingerprint), axis=0)
  res_arr.append(res)
id_map_merged['merged'] = res_arr
id_map_merged.head()

,id,cell_type,sm_name,ohe_col,SMILES,fingerprint,merged
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
1,128,Myeloid cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
2,1,B cells,ABT-199 (GDC-0199),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
3,129,Myeloid cells,ABT-199 (GDC-0199),"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
4,2,B cells,ABT737,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...,"[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."


In [152]:
tmp_id_x = id_map_merged.iloc[:,-1:]
arr = np.empty(len(tmp_id_x), dtype=object)
for i in range(len(tmp_id_x)):
  arr[i] = tmp_id_x.iloc[i,0]
id_X = arr
id_X = np.vstack(id_X)
id_X.shape

(255, 2054)

In [153]:
genes = np.array(de_train.columns[5:18216])
genes[-1]

'ZZEF1'

# Train Full

Split the datasets again:

In [138]:
train_x_split, test_x_split, train_y_split, test_y_split = train_test_split(X, Y, test_size=0.2, random_state=42) 
train_x, val_x, train_y, val_y = train_test_split(train_x_split, train_y_split, test_size=0.2, random_state=42)
    
train_xb, test_xb, train_yb, test_yb = train_test_split(X_b, Y_b, test_size=0.8, random_state=42)
train_xb, val_xb, train_yb, val_yb = train_test_split(train_xb, train_yb, test_size=0.2, random_state=42)
train_xmy, test_xmy, train_ymy, test_ymy = train_test_split(X_my, Y_my, test_size=0.8, random_state=42)
train_xmy, val_xmy, train_ymy, val_ymy = train_test_split(train_xmy, train_ymy, test_size=0.2, random_state=42)
    
X_train = np.concatenate((train_xb, train_xmy, train_x), axis=0)
y_train = np.concatenate((train_yb, train_ymy, train_y), axis=0)
X_val = np.concatenate((val_xb, val_xmy, val_x), axis=0)
y_val = np.concatenate((val_yb, val_ymy, val_y), axis=0) 
X_test = np.concatenate((test_xb, test_xmy, test_x_split), axis=0)
y_test = np.concatenate((test_yb, test_ymy, test_y_split), axis=0)

In [139]:
y_test.shape

(144, 18211)

In [154]:
models_opt = []
    
for i in tqdm(range(Y.shape[1])):
    param = study.best_trial.params
    param['verbosity'] = -1
    dtrain = lgb.Dataset(X_train, label=y_train[:, i])  # Train a model for each output element
    dval = lgb.Dataset(X_val, label=y_val[:, i], reference=dtrain) 
    gbm = lgb.train(param, dtrain, valid_sets=[dval])
    models_opt.append(gbm)
        
pred_y = np.array([model.predict(X_test) for model in models_opt]).T
loss = mrrmse_np(pred_y, y_test)
loss2 = sklearn.metrics.mean_squared_error(pred_y, y_test)
print(loss)
print(loss2)

  0%|          | 0/18211 [00:00<?, ?it/s]/scratch4/mschatz1/hji20/lgbm/lib/python3.8/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/scratch4/mschatz1/hji20/lgbm/lib/python3.8/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
100%|██████████| 18211/18211 [1:01:02<00:00,  4.97it/s]


1.7208496976164644
9.248360776333485


In [156]:
!mkdir trained_models_opt

In [157]:
for i in range(len(models_opt)):
  m = models_opt[i]
  fn = "trained_models_opt/trained_model_" + str(i) + ".txt"
  m.save_model(fn)

In [158]:
id_y_pred = np.array([model.predict(id_X) for model in models_opt]).T
id_y_pred.shape

(255, 18211)

In [159]:
out_fh = open("./submission_opt.csv", 'w')
out_fh.write("id")
for g in genes:
  out_fh.write("," + g)
out_fh.write("\n")

1

In [160]:
ctr = 0
for row in id_y_pred:
    out_fh.write(str(ctr))
    for col in row:
        out_fh.write(',' + str(col))
    out_fh.write('\n')
    ctr += 1

In [161]:
out_fh.close()